In [0]:
from torchvision import transforms, datasets, models
import torchvision
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import tarfile
import requests
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])
valid_transforms = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [0]:
url = "http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz"
file = requests.get(url)
f = open('newfile.tar.gz', 'wb')
f.write(file.content)
f.close()
tar = tarfile.open('newfile.tar.gz', "r:gz")
tar.extractall()
tar.close()
os.remove('newfile.tar.gz')

In [0]:
"""  Set up data directories       """
os.mkdir("datadir")
os.mkdir("datadir/train")
os.mkdir("datadir/valid")
os.mkdir("datadir/test")


for file in os.listdir("101_ObjectCategories/BACKGROUND_Google"):
    os.remove("101_ObjectCategories/BACKGROUND_Google/"+file)
os.rmdir("101_ObjectCategories/BACKGROUND_Google")

for file in os.listdir("101_ObjectCategories/Faces_easy"):
    os.remove("101_ObjectCategories/Faces_easy/"+file)
os.rmdir("101_ObjectCategories/Faces_easy")

def copyfile(filename, fromdir, todir):
    fromfullpath = os.path.join(fromdir, filename)
    tofullpath = os.path.join(todir, filename)
    fromfile = open(fromfullpath, "rb")
    tofile = open(tofullpath, "wb")
    cnt = fromfile.read()
    tofile.write(cnt)
    fromfile.close()
    tofile.close()

pic_dirs = os.listdir("101_ObjectCategories")
new_dirs = ["datadir/train", "datadir/valid", "datadir/test"]

for pic_d in pic_dirs:

    for d in new_dirs:
        os.mkdir(d+"/"+pic_d)
    cur_dir = "101_ObjectCategories/"+pic_d
    files = os.listdir(cur_dir)

    valid_pics = files[:len(files)//4]
    test_pics = files[len(files)//4:len(files)//2]
    train_pics = files[len(files)//2:]
    
    for p in valid_pics:
        copyfile(p, cur_dir, os.path.join("datadir/valid",pic_d))
        os.remove(cur_dir+"/"+p)

    for p in test_pics:
        copyfile(p, cur_dir, os.path.join("datadir/test",pic_d))
        os.remove(cur_dir+"/"+p)

    for p in train_pics:
        copyfile(p, cur_dir, "datadir/train/"+pic_d)
        os.remove(cur_dir+"/"+p)
    
    os.rmdir(cur_dir)

os.rmdir("101_ObjectCategories")

In [4]:
traindir = "datadir/train"
validdir = "datadir/valid"
testdir = "datadir/test"
batchS = 128
# Datasets from folders
train_data = datasets.ImageFolder(root=traindir, transform=train_transforms)
valid_data = datasets.ImageFolder(root=validdir, transform=valid_transforms)
test_data = datasets.ImageFolder(root=testdir, transform=valid_transforms)

# Dataloader iterators, make sure to shuffle
train_dataloader = DataLoader(train_data, batch_size=batchS, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=batchS, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batchS, shuffle=False)

trainiter = iter(train_dataloader)
features, labels = next(trainiter)
features.shape, labels.shape

(torch.Size([128, 3, 224, 224]), torch.Size([128]))

In [5]:
model = models.vgg16(pretrained=True)

# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

n_classes=101
n_inputs = 4096
# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

model.classifier

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:10<00:00, 52.5MB/s]


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=101, bias=True)
    (4): LogSoftmax()
  )
)

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

135,335,333 total parameters.
1,074,789 training parameters.


I don't know why I have more parameters. In the guide they had 135,335,076 total parameters and 1,074,532 training parameters.

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Distribute across 2 gpus
model = nn.DataParallel(model)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [8]:
n_epochs = 10
# Early stopping details
n_epochs_stop = 5
min_val_loss = np.Inf
epochs_no_improve = 0
os.mkdir("saved_model")
checkpoint_path = "saved_model/the_model.pt"

for epoch in range(n_epochs):
    val_loss = 0

    for data, targets in train_dataloader:
        data = data.to(device)
        targets = targets.to(device)
        out = model(data)
        # Calculate loss
        loss = criterion(out, targets)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()

    for data, targets in valid_dataloader:
        data = data.to(device)
        targets = targets.to(device) 
        out = model(data)
        loss = criterion(out, targets)
        val_loss += loss

    # Average validation loss
    val_loss = val_loss / len(valid_dataloader)
    print("epoch: {}   validation loss: {}".format(epoch, val_loss))

    # If the validation loss is at a minimum
    if val_loss < min_val_loss:
        torch.save(model, checkpoint_path)
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
        # Check early stopping condition
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            # Load the best model
            model = torch.load(checkpoint_path)

epoch: 0   validation loss: 1.8340507745742798


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/

epoch: 1   validation loss: 1.928882360458374
epoch: 2   validation loss: 2.316047191619873
epoch: 3   validation loss: 2.410613775253296
epoch: 4   validation loss: 2.5781357288360596
epoch: 5   validation loss: 2.3955328464508057
Early stopping!
epoch: 6   validation loss: 1.8338189125061035
epoch: 7   validation loss: 1.8251739740371704
epoch: 8   validation loss: 1.8295753002166748
epoch: 9   validation loss: 1.8364481925964355


In [9]:
correct = 0
topk_correct = 0
all = 0
model.eval()
for data, targets in test_dataloader:
    data, targets = data.to(device), targets.to(device)
    log_ps = model(data)
    # Convert to probabilities
    ps = torch.exp(log_ps).detach()

    top_5_ps, top_5_classes = ps.topk(5, dim=1)
    for i in range(top_5_classes.shape[0]):
        five_classes = top_5_classes[i].cpu().numpy()
        the_target = targets[i].item()
        fl = False
        for cl in five_classes:
            if the_target == cl:
                fl = True
                break
        if fl:
            topk_correct += 1
    
    pred = torch.max(ps, dim=1).indices
    equals = pred==targets
    correct +=  torch.sum(equals).cpu().item()
    all += len(equals)
    
accuracy = correct / all
print("Accuracy: ", accuracy)
print("Top 5 accuracy: ", topk_correct/all)

Accuracy:  0.6637764932562621
Top 5 accuracy:  0.8742774566473989


I have much lower accuracy than they had in the guide:(

In [10]:
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))
total = 0
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(images.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            total += 1

classes = np.arange(0, 100)
for i in range(100):
    print('Accuracy of class %2s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of class  0 : 99 %
Accuracy of class  1 : 100 %
Accuracy of class  2 : 100 %
Accuracy of class  3 : 92 %
Accuracy of class  4 : 98 %
Accuracy of class  5 :  0 %
Accuracy of class  6 :  0 %
Accuracy of class  7 : 41 %
Accuracy of class  8 : 64 %
Accuracy of class  9 : 58 %
Accuracy of class 10 : 25 %
Accuracy of class 11 : 96 %
Accuracy of class 12 :  0 %
Accuracy of class 13 :  0 %
Accuracy of class 14 :  0 %
Accuracy of class 15 : 73 %
Accuracy of class 16 : 92 %
Accuracy of class 17 :  0 %
Accuracy of class 18 : 100 %
Accuracy of class 19 :  0 %
Accuracy of class 20 : 33 %
Accuracy of class 21 : 31 %
Accuracy of class 22 : 74 %
Accuracy of class 23 : 58 %
Accuracy of class 24 : 64 %
Accuracy of class 25 : 16 %
Accuracy of class 26 :  0 %
Accuracy of class 27 :  7 %
Accuracy of class 28 :  7 %
Accuracy of class 29 : 85 %
Accuracy of class 30 : 58 %
Accuracy of class 31 : 76 %
Accuracy of class 32 : 18 %
Accuracy of class 33 : 41 %
Accuracy of class 34 : 47 %
Accuracy of class

How to get class names? (to print: "accuracy of crocodile" instead of "accuracy of class 56")